In [312]:
import numpy as np
from scipy.linalg import eigh_tridiagonal
from itertools import combinations

In [354]:
class poly:

    def __init__(self, n, coefs):
        if (n != len(coefs)):
            print(f'error {n=} != {len(coefs)}')
        self.n = n
        self.coefs = list(coefs)

    def __len__(self):
        return self.n

    def __getitem__(self, i):
        return self.coefs[i]

    def __call__(self, x):
        res = self.coefs[0]
        running = x
        for i in range(1, self.n + 1):
            res += self.coefs[i] * running
            running *= x
        return res

    def __str__(self):
        s = ''
        for i in range(self.n):
            s += f'{self[self.n-i-1]:.2f}*x^{self.n-i-1} + '
        return s[:-3]

    def __mul__(self, other):
        if isinstance(other, (int, float, complex)):
            new_coef = []
            for i in range(self.n):
                new_coef.append(self[i] * other)
            return poly(self.n, new_coef)
        elif isinstance(other, self.__class__):
            pass
        else:
            print(f'dont know how to multiple {type(other)} :(')
        return 0

    def times_x(self):
        new_coef = [0] + self.coefs
        return poly(self.n + 1, new_coef)

    def __add__(self, other):
        if isinstance(other, self.__class__):
            new_coef = self.coefs.copy()
            self_longer = False
            for i in range(self.n):
                try:
                    new_coef[i] += other[i]
                except IndexError:
                    self_longer = True
                    break
            if not self_longer:
                for i in range(len(new_coef), len(other)):
                    new_coef.append(other[i])
            return poly(len(new_coef), new_coef)
        else:
            print(f'dont know how to add {type(other)} :(')
        return 0

    def last(self):
        return self[self.n - 1]

In [355]:
a = poly(3, [1, 2, 3])
b = poly(5, [1, 2, 3, 4, 5])
print(a + b)
print(b + a)

5.00*x^4 + 4.00*x^3 + 6.00*x^2 + 4.00*x^1 + 2.00*x^0
5.00*x^4 + 4.00*x^3 + 6.00*x^2 + 4.00*x^1 + 2.00*x^0


In [356]:
def deg_integral(coef, n, a, b):
    '''integral from a to b of coef * x**n'''
    return (b**(n + 1) - a**(n + 1)) / (n + 1) * coef

In [357]:
def scalar_prod(p, q, a, b):
    res = 0
    for i in range(len(p)):
        for j in range(len(q)):
            res += deg_integral(p[i] * q[j], i + j, a, b)
    return res

In [358]:
def gramm_matrix(n, a, b):
    g = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            g[i, j] = deg_integral(1, i + j, a, b)
    return g

In [359]:
s = np.linalg.inv(np.linalg.cholesky(gramm_matrix(3, 0, 1)))
for i in range(3):
    print(s[i][:i+1])

[1.]
[-1.73205081  3.46410162]
[  2.23606798 -13.41640786  13.41640786]


In [360]:
def orthogonal_polynomials_cholesky(n, a, b, verbose=True):
    polys = []
    S = np.linalg.inv(np.linalg.cholesky(gramm_matrix(n, a, b)))
    for i in range(n):
        coeffs = S[i][:i+1]
        polys.append(poly(i + 1, coeffs))
        if verbose:
            print(polys[i])

    if verbose:
        for i in range(n):
            for j in range(i + 1, n):
                print(f'{i}x{j}: ', scalar_prod(polys[i], polys[j], a, b))
    return polys

In [361]:
orthogonal_polynomials_cholesky(5, 0, 1)

1.00*x^0
3.46*x^1 + -1.73*x^0
13.42*x^2 + -13.42*x^1 + 2.24*x^0
52.92*x^3 + -79.37*x^2 + 31.75*x^1 + -2.65*x^0
210.00*x^4 + -420.00*x^3 + 270.00*x^2 + -60.00*x^1 + 3.00*x^0
0x1:  0.0
0x2:  0.0
0x3:  0.0
0x4:  -2.1316282072803006e-14
1x2:  0.0
1x3:  7.105427357601002e-15
1x4:  2.842170943040401e-14
2x3:  0.0
2x4:  3.410605131648481e-13
3x4:  -4.547473508864641e-13


In [362]:
for i in range(100):
    try:
        orthogonal_polynomials_cholesky(i, 0, 1, False)
    except Exception as e:
        print(i, str(e), ':(')
        break

14 Matrix is not positive definite :(


In [363]:
for i in range(100):
    try:
        orthogonal_polynomials_cholesky(i, -1, 1, False)
    except Exception as e:
        print(i, str(e), ':(')
        break

26 Matrix is not positive definite :(


In [364]:
def orthogonal_polynomials_recur(n, a, b, verbose=True):
    polys = []
    l0 = poly(1, [1])
    alpha = deg_integral(1, 1, a, b)
    polys.append(l0)
    l1 = l0.times_x() + l0 * (-alpha)
    l1 = l1 * (1 / np.sqrt(scalar_prod(l1, l1, a, b)))
    polys.append(l1)
    beta = scalar_prod(l0.times_x(), l1, a, b)
    if verbose:
        print(l0)
        print(l1)
    for i in range(2, n):
        ln = polys[i - 1]
        alpha = scalar_prod(ln.times_x(), ln, a, b)
        l = ln.times_x() + ln * (-alpha)
        l += polys[i - 2] * (-beta)
        l *= 1 / np.sqrt(scalar_prod(l, l, a, b))
        beta = scalar_prod(ln.times_x(), l, a, b)
        polys.append(l)
        if verbose:
            print(l)

    if verbose:
        for i in range(n):
            for j in range(i + 1, n):
                print(f'{i}x{j}: ', scalar_prod(polys[i], polys[j], a, b))
    return polys

In [365]:
orthogonal_polynomials_recur(5, 0, 1)

1.00*x^0
3.46*x^1 + -1.73*x^0
13.42*x^2 + -13.42*x^1 + 2.24*x^0
52.92*x^3 + -79.37*x^2 + 31.75*x^1 + -2.65*x^0
210.00*x^4 + -420.00*x^3 + 270.00*x^2 + -60.00*x^1 + 3.00*x^0
0x1:  0.0
0x2:  1.7763568394002505e-15
0x3:  -5.329070518200751e-15
0x4:  -1.4210854715202004e-14
1x2:  0.0
1x3:  -7.105427357601002e-15
1x4:  4.263256414560601e-14
2x3:  2.842170943040401e-14
2x4:  -1.1368683772161603e-13
3x4:  4.547473508864641e-13


In [366]:
def poly_from_roots(roots):
    coefs = [0]
    n = len(roots)
    for i in range(n - 1, 0, -1):
        s = 0
        for comb in combinations(roots, i):
            coef = 1
            for el in comb:
                coef *= el
            s += coef
        s = -s if i % 2 == 1 else s
        coefs.append(s)
    
    free = 1
    for i in roots:
        free *= i
    coefs[0] = -free if n % 2 == 1 else free
    coefs.append(1)
    return poly(n + 1, coefs)

In [367]:
print(poly_from_roots([2, 3, 4, 5]))

1.00*x^4 + -14.00*x^3 + 71.00*x^2 + -154.00*x^1 + 120.00*x^0


In [392]:
def orthogonal_polynomials_roots(n, a, b, verbose=True):
    alphas = []
    betas = []
    polys = []
    l0 = poly(1, [1])
    alpha = deg_integral(1, 1, a, b)
    polys.append(l0)
    l1 = l0.times_x() + l0 * (-alpha)
    l1 = l1 * (1 / np.sqrt(scalar_prod(l1, l1, a, b)))
    polys.append(l1)
    beta = scalar_prod(l0.times_x(), l1, a, b)
    alphas.append(alpha)
    betas.append(beta)
    alphas.append(scalar_prod(l1, l1.times_x(), a, b))
    if verbose:
        print(l0)
        print(l1)
        print()
    for i in range(2, n):
        roots = eigh_tridiagonal(alphas, betas, eigvals_only=True)
        l_i = poly_from_roots(roots)
        l_i = l_i * (1 / np.sqrt(scalar_prod(l_i, l_i, a, b)))
        alphas.append(scalar_prod(l_i, l_i.times_x(), a, b))
        betas.append(scalar_prod(l_i, polys[i - 1].times_x(), a, b))
        polys.append(l_i)
        if verbose:
            print(roots)
            print(l_i)
            print()

    if verbose:
        for i in range(n):
            for j in range(i + 1, n):
                print(f'{i}x{j}: ', scalar_prod(polys[i], polys[j], a, b))

In [397]:
orthogonal_polynomials_roots(5, 0, 1, verbose=True)

1.00*x^0
3.46*x^1 + -1.73*x^0

[0.21132487 0.78867513]
13.42*x^2 + -13.42*x^1 + 2.24*x^0

[0.11270167 0.5        0.88729833]
52.92*x^3 + -79.37*x^2 + 31.75*x^1 + -2.65*x^0

[0.06943184 0.33000948 0.66999052 0.93056816]
210.00*x^4 + -420.00*x^3 + 270.00*x^2 + -60.00*x^1 + 3.00*x^0

0x1:  0.0
0x2:  1.7763568394002505e-15
0x3:  -1.7763568394002505e-15
0x4:  -2.842170943040401e-14
1x2:  0.0
1x3:  -2.1316282072803006e-14
1x4:  -2.842170943040401e-14
2x3:  -2.842170943040401e-14
2x4:  -2.8421709430404007e-13
3x4:  -1.3642420526593924e-12
